In [3]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.neighbors import DistanceMetric
import numpy as np
import scipy
from pathlib import Path  
from nilearn import plotting

In [4]:
# Subject variables 
subj = '06'
sess = 'iemu'
task = 'film'
acq = 'clinical'
run = 1

In [5]:
df = pd.read_csv('/home/jovyan/bipolar_ref_ieeg/coord_data.csv') #csv file from Azeezat

dist = DistanceMetric.get_metric('euclidean')
tmp = scipy.spatial.distance.cdist(df[['x_coord','y_coord','z_coord']].to_numpy(), df[['x_coord','y_coord','z_coord']].to_numpy())

dist_df = pd.DataFrame(tmp)
dist_df["Channel_Name"] = df.Channel_Name
dist_df = dist_df.set_index("Channel_Name")
dist_df.columns = df.Channel_Name

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/bipolar_ref_ieeg/coord_data.csv'

In [6]:
# get the unique pad types 
df["PadType"] = df['Channel_Name'].str.replace('\d+', '')
uniquePadTypes = pd.unique(df["PadType"])

substring = "+"
makePlots = False
horizontial = False

for pad in uniquePadTypes:
    padIDX = df["PadType"] == pad
    df2 = df[padIDX]
    df2 = df2.set_index("Channel_Name")
    padDistances = dist_df.loc[padIDX.tolist(),padIDX.tolist()]

    if substring in pad:
        print(pad + " is not an iEEG signal")
        continue

    # COMPUTE THRESHOLD:
    NN1 = np.diagonal(padDistances,offset=1) #get all of the first nearest neighbors
    # hist, bin_edges = np.histogram(NN1)    
    # thresh = bin_edges[0] + NN1.std()/2
    thresh = NN1.mean() + NN1.std()/2 
    
    
   

    # APPLY THRESHOLD TO HORIZONTIAL OR VERTICAL DIRECTIONS
    if horizontial:
        padDistanceRowOne = (padDistances.iloc[:,0] < thresh)
        numTrue = padDistanceRowOne.sum()
        if numTrue >= 3:
            trueIDX = (padDistanceRowOne[padDistanceRowOne==True].index.tolist()[2])
            padDistanceRowOne = padDistanceRowOne.reset_index()
            offsetIndex = int(padDistanceRowOne.index[padDistanceRowOne.Channel_Name == trueIDX].to_numpy())
            NN1 = np.diagonal(padDistances,offset=offsetIndex) #get all of the first nearest neighbors
        elif numTrue < 3:
            offsetIndex = 1
            NN1 = np.diagonal(padDistances,offset=offsetIndex) #get all of the first nearest neighbors
        tag = "horizontial"
    elif not horizontial:
        offsetIndex = 1
        NN1 = np.diagonal(padDistances,offset=offsetIndex) #get all of the first nearest neighbors
        tag = "vertical"

    # Apply the threshold to the distance values
    NN1_thresh = (NN1 < thresh) * NN1

    # Indexing
    padLabels = list(padDistances.index.values)
    matrix = [[x + y for x in padLabels] for y in padLabels]
    padMatrix = pd.DataFrame(matrix)

    NN1_Labels = np.diagonal(padMatrix,offset=offsetIndex) #get all of the first nearest neighbors labels

    e1_labels, e2_labels = zip(*(s.rsplit(pad[0],1) for s in NN1_Labels))
    e1_labels = list(e1_labels)
    e2_labels = [pad[0] + s for s in e2_labels]

    NN_df = pd.DataFrame(e1_labels, columns = ['e1_Labels'])
    NN_df['e2_Labels']= e2_labels
    NN_df["Distances"] = NN1_thresh

    # Include x,y,z coordinates to the NN dataframe
    NN_df = pd.merge(NN_df, df2.loc[:,['x_coord','y_coord','z_coord']], how='left', left_on='e1_Labels', right_on='Channel_Name')
    NN_df = pd.merge(NN_df, df2.loc[:,['x_coord','y_coord','z_coord']], how='left', left_on='e2_Labels', right_on='Channel_Name',suffixes=('_e1','_e2'))

    # Remove distances that are equal to zero
    NN_df = NN_df.drop(NN_df[NN_df.Distances ==0].index)
    
    # display and save
    display(NN_df)
    filepath = Path('/home/jovyan/bipolar_ref_ieeg/'+subj+'_'+sess+'_'+task+'_'+str(run)+'_'+pad+'_'+tag+'_bipolarRef.csv')
    NN_df.to_csv(filepath)
    

NameError: name 'df' is not defined